# Safe LA: Enhancing Traffic Safety Through Data-Driven Insights 



In [79]:
#Cell for all imports
import pandas as pd
import numpy as np
import sqlite3
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
import plotly.express as px
import re
import os
import zipfile

In [80]:
#Load data
df_old = pd.read_csv('Traffic_Collision_Data_from_2010_to_Present.csv')

df_old.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Code,Premise Description,Address,Cross Street,Location
0,190319651,08/24/2019,08/24/2019,450,3,Southwest,356,997,TRAFFIC COLLISION,3036 3004 3026 3101 4003,22.0,M,H,101.0,STREET,JEFFERSON BL,NORMANDIE AV,"(34.0255, -118.3002)"
1,190319680,08/30/2019,08/30/2019,2320,3,Southwest,355,997,TRAFFIC COLLISION,3037 3006 3028 3030 3039 3101 4003,30.0,F,H,101.0,STREET,JEFFERSON BL,W WESTERN,"(34.0256, -118.3089)"
2,190413769,08/25/2019,08/25/2019,545,4,Hollenbeck,422,997,TRAFFIC COLLISION,3101 3401 3701 3006 3030,NaN,M,X,101.0,STREET,N BROADWAY,W EASTLAKE AV,"(34.0738, -118.2078)"
3,190127578,11/20/2019,11/20/2019,350,1,Central,128,997,TRAFFIC COLLISION,0605 3101 3401 3701 3011 3034,21.0,M,H,101.0,STREET,1ST,CENTRAL,"(34.0492, -118.2391)"
4,190319695,08/30/2019,08/30/2019,2100,3,Southwest,374,997,TRAFFIC COLLISION,0605 4025 3037 3004 3025 3101,49.0,M,B,101.0,STREET,MARTIN LUTHER KING JR,ARLINGTON AV,"(34.0108, -118.3182)"


In [81]:
#Filtering the occurence date to just 2023 and 2024
df = df_old[df_old['Date Occurred'].str.contains('2023|2024', na=False)]

print(df.shape)

(33114, 18)


In [82]:
print("Columns - " ,df.columns)  # See what columns exist
print("No of columns - ", len(df.columns))  # Total number of columns

Columns -  Index(['DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred',
       'Area ID', 'Area Name', 'Reporting District', 'Crime Code',
       'Crime Code Description', 'MO Codes', 'Victim Age', 'Victim Sex',
       'Victim Descent', 'Premise Code', 'Premise Description', 'Address',
       'Cross Street', 'Location'],
      dtype='object')
No of columns -  18


Some of the columns are anonymized or are redundant with an existing column and will have no impact on our analysis, which we will be removing immediately. This includes 'DR Number', 'Date Reported', 'Area ID', 'Reporting District', 'Crime Code', 'Crime Code Description', and 'Premise Code'.

In [83]:
#Dropping previously mentioned columns

df = df.drop(columns=['Date Reported', 'Area ID', 'Reporting District', 'Crime Code', 'Crime Code Description', 'Premise Code'])

df.head()


,DR Number,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,Address,Cross Street,Location
846,231314324,07/22/2023,800,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26.0,M,H,STREET,WASHINGTON,OLIVE,"(34.0325, -118.2676)"
19678,240406645,03/10/2024,1900,Hollenbeck,0605 3030 4024,31.0,M,C,STREET,200 W AVENUE 26,NaN,"(34.0786, -118.2174)"
44978,230812188,07/19/2023,1230,West LA,3004 3028 3033 3034 3037 3030,31.0,M,B,STREET,SAWTELLE BL,OLYMPIC BL,"(34.0382, -118.4412)"
45031,241307843,03/27/2024,2340,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25.0,M,O,SIDEWALK,ADAMS,GRAND,"(34.0265, -118.2723)"
45311,241214094,08/05/2024,835,77th Street,NaN,30.0,M,H,STREET,FLORENCE,GRAND,"(33.9747, -118.2803)"


In [84]:
print("Columns - " ,df.columns)  # See what columns exist
print("No of columns - ", len(df.columns))  # Total number of columns

Columns -  Index(['DR Number', 'Date Occurred', 'Time Occurred', 'Area Name', 'MO Codes',
       'Victim Age', 'Victim Sex', 'Victim Descent', 'Premise Description',
       'Address', 'Cross Street', 'Location'],
      dtype='object')
No of columns -  12


Additionally, we will be checking data types of the columns we will be using to ensure they are in the right format, if not we will change them.

In [85]:
#Checking the data type of each column
data_types = df.dtypes
data_types

,0
DR Number,int64
Date Occurred,object
Time Occurred,int64
Area Name,object
MO Codes,object
Victim Age,float64
Victim Sex,object
Victim Descent,object
Premise Description,object
Address,object


In [86]:
# The victim age column should be an integer as opposed to a float, we will change the data type of that column
# Date occured and time occured will be changed in a later section
df['Victim Age'] = pd.to_numeric(df['Victim Age'], errors='coerce').astype('Int64')

The MO codes each have a definition to them, we will first seperate them by the spaces in the string, and then try to remap them into their definitions.

In [87]:
# create new columns with a seperate mo code in each
mo_columns = df["MO Codes"].astype(str).str.split(expand=True)

mo_columns

,0,1,2,3,4,5,6,7,8,9
846,3401,3701,3036,0605,3003,3024,3033,3035,3101,4024
19678,0605,3030,4024,None,None,None,None,None,None,None
44978,3004,3028,3033,3034,3037,3030,None,None,None,None
45031,0605,3011,3028,3033,3034,3036,3101,4013,3401,3701
45311,nan,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
619942,3004,3028,3030,3101,4015,None,None,None,None,None
619943,3004,3028,4026,3037,3101,None,None,None,None,None
619944,3004,3026,3037,3038,4027,3035,3101,3401,3701,None
619945,3101,3011,3028,4026,3030,3037,None,None,None,None


In [88]:
# Converts the table into a single entry
mo_codes = mo_columns.values.ravel()

In [89]:
# only gets unique values
unique_mo_codes = pd.unique(mo_codes)

In [90]:
#print the number of mo codes that show up
n_unique = len(unique_mo_codes)

print(n_unique)

151


In [91]:
# prints list
print(unique_mo_codes)

['3401' '3701' '3036' '0605' '3003' '3024' '3033' '3035' '3101' '4024'
 '3030' None '3004' '3028' '3034' '3037' '3011' '4013' 'nan' '3026' '3029'
 '3025' '3039' '2004' '0910' '3008' '3006' '4009' '4002' '4025' '0947'
 '3002' '3602' '3032' '4003' '3038' '3102' '4004' '4005' '1402' '3009'
 '4027' '4026' '4010' '3014' '4011' '4012' '4001' '4014' '4015' '4017'
 '4016' '1218' '3013' '3027' '4019' '4021' '3103' '3064' '4008' '4007'
 '3104' '3012' '4018' '4020' '4006' '3001' '3040' '3020' '3007' '3010'
 '1407' '1419' '3005' '0906' '1816' '3601' '3501' '3063' '3062' '1822'
 '1609' '1309' '3016' '0604' '3018' '3015' '2018' '1300' '0416' '1312'
 '2029' '0358' '3022' '2028' '0546' '0907' '3023' '1025' '2021' '2002'
 '0918' '1501' '1004' '1303' '0601' '3017' '2101' '2042' '3019' '2024'
 '1217' '0342' '1409' '0424' '0336' '2038' '0216' '0325' '0329' '3603'
 '0443' '0913' '0422' '0432' '0352' '1420' '2303' '3021' '0400' '0328'
 '1313' '0425' '1026' '3801' '2030' '0401' '1021' '0946' '1403' '0919'
 '

In [92]:
MO_Codes_Ref = pd.read_csv('MO_Codes_Ref.csv')

MO_Codes_Ref['Description'] = MO_Codes_Ref['Description'].apply(
    lambda x: 'Public Transit' if 'Public Transit (Metrolink/Train Station,Metro Rail Red,Line Subway Station, Metro Rail Blue Line Station,adjacent transit parking lots, tracks or tunnels MTA(RTD), and other municipal lines.' in str(x) else x
)
MO_Codes_Ref

,MO Code,Description
0,100,Suspect Impersonate
1,101,Aid victim
2,102,Blind
3,103,Crippled
4,104,Customer
...,...,...
772,4021,T/C - Topanga
773,4024,T/C - Central Traffic (CTD)
774,4025,T/C - South Traffic (STD)
775,4026,T/C - Valley Traffic (VTD)


In [93]:
unique_mo_df = pd.DataFrame(unique_mo_codes, columns=['mo_code'])

# Display the DataFrame
print(unique_mo_df)

    mo_code
0      3401
1      3701
2      3036
3      0605
4      3003
..      ...
146    1903
147    1605
148    0505
149    2014
150    0417

[151 rows x 1 columns]


In [94]:
# Creating a new SQLite database
conn = sqlite3.connect('temp_database.db')

# Create a cursor object
cursor = conn.cursor()

In [95]:
# Convert the unique_mo_df and MO_Codes_Ref into SQL Tables
unique_mo_df.to_sql('UMO', conn, if_exists='replace', index=False)
MO_Codes_Ref.to_sql('RMO', conn, if_exists='replace', index=False)


777

In [96]:
#join the tables to get description for each unique MO

#write the query to be executed
query = """
SELECT t1.*, t2.Description
FROM UMO t1 LEFT JOIN RMO t2
ON t1.mo_code = t2.[MO Code];
"""

# query result as a pd dataframe
df_result = pd.read_sql_query(query, conn)

# Close connection to sqlite database that we created
conn.close() #rerun the connection query if need to run this section of code again

In [97]:
print(df_result)

    mo_code                                        Description
0      3401                            T/C - Type of Collision
1      3701                 T/C - Movement Preceding Collision
2      3036                        T/C - At Intersection - Yes
3      0605          Traffic Accident/Traffic related incident
4      3003                                   T/C - Veh vs Ped
..      ...                                                ...
146    1903  Cyberstalking (Stalking using internet to comm...
147    1605                                 Lock slip/key/pick
148    0505                           Forced to fondle suspect
149    2014                       Suspect was Uber/Lyft driver
150    0417                                             Kicked

[151 rows x 2 columns]


In [98]:
# get rows where mo_code is null
empties = df_result[df_result['mo_code'].isna()]

print(empties)

   mo_code Description
11    None        None


In [99]:
# Drop the rows which are null
df_result = df_result.drop([11], axis=0).reset_index(drop=True)

# String matching MO Codes

In [100]:
MO_Code_Desc = df_result

print(MO_Code_Desc)

    mo_code                                        Description
0      3401                            T/C - Type of Collision
1      3701                 T/C - Movement Preceding Collision
2      3036                        T/C - At Intersection - Yes
3      0605          Traffic Accident/Traffic related incident
4      3003                                   T/C - Veh vs Ped
..      ...                                                ...
145    1903  Cyberstalking (Stalking using internet to comm...
146    1605                                 Lock slip/key/pick
147    0505                           Forced to fondle suspect
148    2014                       Suspect was Uber/Lyft driver
149    0417                                             Kicked

[150 rows x 2 columns]


In [101]:
# map the mo codes and the descriptions
map = dict(zip(MO_Code_Desc['mo_code'], MO_Code_Desc['Description']))

# replace all values in the mo_code tables
mo_columns = mo_columns.replace(map)

mo_columns

,0,1,2,3,4,5,6,7,8,9
846,T/C - Type of Collision,T/C - Movement Preceding Collision,T/C - At Intersection - Yes,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD)
19678,Traffic Accident/Traffic related incident,T/C - Hit and Run Misd,T/C - Central Traffic (CTD),None,None,None,None,None,None,None
44978,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,None,None,None,None
45031,Traffic Accident/Traffic related incident,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision
45311,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
619942,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Hit and Run Misd,T/C - PCF (A) In the Narrative,T/C - North Hollywood,None,None,None,None,None
619943,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Valley Traffic (VTD),T/C - At Intersection - No,T/C - PCF (A) In the Narrative,None,None,None,None,None
619944,T/C - Veh vs Veh,T/C - (C) Complaint of Injury,T/C - At Intersection - No,T/C - DUI Felony,T/C - West Traffic (WTD),T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Type of Collision,T/C - Movement Preceding Collision,None
619945,T/C - PCF (A) In the Narrative,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Valley Traffic (VTD),T/C - Hit and Run Misd,T/C - At Intersection - No,None,None,None,None


In [102]:
# Merge the two tables
df = pd.concat([df, mo_columns], axis=1)

df.head()


,DR Number,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,Address,...,0,1,2,3,4,5,6,7,8,9
846,231314324,07/22/2023,800,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,WASHINGTON,...,T/C - Type of Collision,T/C - Movement Preceding Collision,T/C - At Intersection - Yes,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD)
19678,240406645,03/10/2024,1900,Hollenbeck,0605 3030 4024,31,M,C,STREET,200 W AVENUE 26,...,Traffic Accident/Traffic related incident,T/C - Hit and Run Misd,T/C - Central Traffic (CTD),None,None,None,None,None,None,None
44978,230812188,07/19/2023,1230,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,SAWTELLE BL,...,T/C - Veh vs Veh,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,None,None,None,None
45031,241307843,03/27/2024,2340,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,ADAMS,...,Traffic Accident/Traffic related incident,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision
45311,241214094,08/05/2024,835,77th Street,NaN,30,M,H,STREET,FLORENCE,...,None,None,None,None,None,None,None,None,None,None


# Location Data

We will create a new column that contains the intersection of the address and cross street to get a better picture of high risk areas.

The code creates a function that combines the two, however important to note that intersections may show up twice in situations where the address and cross street are changed.

For example: 5th avenue & 6th street,
             6th street & 5th avenue are functionally the same.

We will avoid this by ordering the intersection in alphabetical order so the first street amongst the two will always take priority.
`


In [103]:
# Create a function that will make the intersection
def intersection(i):
    streets = [i["Address"], i["Cross Street"]]
    streets = [x for x in streets if pd.notna(x)]
    return "&".join(sorted(streets))

# Create a function that will return a single white space in place of any number of white spaces
# Due to entries include long segments of white space
def no_duplicate_spaces(i):
    return " ".join(re.split(r"\s+", i.strip()))

# removing white spaces from the two columns
df["Address"] = df["Address"].str.strip()
df["Cross Street"] = df["Cross Street"].str.strip()

# Apply the functions to the DataFrame
# First one makes new column for the joining of intersection

df["Intersection"] = df.apply(intersection, axis=1)
df["Intersection"] = df["Intersection"].apply(no_duplicate_spaces)

In [104]:
df = df.drop(columns=['Address', 'Cross Street'])

df.head()

,DR Number,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,Location,...,1,2,3,4,5,6,7,8,9,Intersection
846,231314324,07/22/2023,800,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,"(34.0325, -118.2676)",...,T/C - Movement Preceding Collision,T/C - At Intersection - Yes,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),OLIVE&WASHINGTON
19678,240406645,03/10/2024,1900,Hollenbeck,0605 3030 4024,31,M,C,STREET,"(34.0786, -118.2174)",...,T/C - Hit and Run Misd,T/C - Central Traffic (CTD),None,None,None,None,None,None,None,200 W AVENUE 26
44978,230812188,07/19/2023,1230,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,"(34.0382, -118.4412)",...,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,None,None,None,None,OLYMPIC BL&SAWTELLE BL
45031,241307843,03/27/2024,2340,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,"(34.0265, -118.2723)",...,T/C - Veh vs Fixed Object,T/C - (N) Non Injury,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision,ADAMS&GRAND
45311,241214094,08/05/2024,835,77th Street,NaN,30,M,H,STREET,"(33.9747, -118.2803)",...,None,None,None,None,None,None,None,None,None,FLORENCE&GRAND


We will also be utilizing latitude and longitudinal data in our analysis, we will be splitting the data into two seperate columns.

In [105]:
# Remove brackets
df["Location"]= df["Location"].str.strip("()")

# seperating the data into latitude and longitude
df[["Latitude", "Longitude"]] = df["Location"].str.split(",", expand=True)

# removing white spaces
df["Latitude"] = df["Latitude"].str.strip().astype(float)
df["Longitude"] = df["Longitude"].str.strip().astype(float)

df.drop("Location", axis=1, inplace=True)

df.head()

,DR Number,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,0,...,3,4,5,6,7,8,9,Intersection,Latitude,Longitude
846,231314324,07/22/2023,800,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,T/C - Type of Collision,...,Traffic Accident/Traffic related incident,T/C - Veh vs Ped,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),OLIVE&WASHINGTON,34.0325,-118.2676
19678,240406645,03/10/2024,1900,Hollenbeck,0605 3030 4024,31,M,C,STREET,Traffic Accident/Traffic related incident,...,None,None,None,None,None,None,None,200 W AVENUE 26,34.0786,-118.2174
44978,230812188,07/19/2023,1230,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,T/C - Veh vs Veh,...,T/C - City Property Involved - Yes,T/C - At Intersection - No,T/C - Hit and Run Misd,None,None,None,None,OLYMPIC BL&SAWTELLE BL,34.0382,-118.4412
45031,241307843,03/27/2024,2340,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,Traffic Accident/Traffic related incident,...,T/C - Private Property - No,T/C - City Property Involved - Yes,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision,ADAMS&GRAND,34.0265,-118.2723
45311,241214094,08/05/2024,835,77th Street,NaN,30,M,H,STREET,None,...,None,None,None,None,None,None,None,FLORENCE&GRAND,33.9747,-118.2803


In [106]:
# shortening some premise descriptions and removing special characters
df["Premise Description"] = df["Premise Description"].replace({"TRAIN, OTHER THAN MTA (ALSO QUERY 809/810/811)": "TRAIN DEPOT",
                                                               "TRAIN DEPOT/TERMINAL, OTHER THAN MTA": "TRAIN DEPOT",
                                                               "MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)": "MULTI UNIT DWELLING"})

# Date and Time

In [107]:
#convert Date Occurred to datetime
df['Date Occurred'] = pd.to_datetime(df['Date Occurred'], format='%m/%d/%Y')

In [108]:
#convert Time Occurred to datetime
df['Time Occurred'] = pd.to_datetime(df['Time Occurred'].astype(str).str.zfill(4), format='%H%M').dt.strftime('%H:%M')

In [109]:
# Takes the first 2 digits of the time and creates a new column
df["Hour"] = df["Time Occurred"].astype(str).str[:2].astype(int)

# Age, Sex and Descent

In [110]:
#fill blanks with unknown/X for Victim Sex
for x in df.index:
  if df.loc[x, "Victim Sex"] == "F" or df.loc[x, "Victim Sex"] == "M":
    continue
  else:
    df.loc[x, "Victim Sex"] = "X"

In [111]:
#fill blanks with unknown/X for Victim Descent
Descent_codes = ["A","B","C","D","F","G","H","I","J","K","L","O","P","S","U","V","W","Z"]

for x in df.index:
  if df.loc[x, "Victim Descent"] in Descent_codes:
    continue
  else:
    df.loc[x, "Victim Descent"] = "X"

In [112]:
#recode age
print("Max Age - ",df['Victim Age'].max())
print("Min Age - ",df['Victim Age'].min())
print(pd.unique(df['Victim Age']))

Max Age -  99
Min Age -  10
<IntegerArray>
[  26,   31,   25,   30,   32, <NA>,   35,   24,   13,   23,   29,   28,   21,
   57,   51,   38,   33,   99,   22,   43,   20,   34,   72,   17,   60,   45,
   63,   59,   27,   64,   55,   48,   80,   65,   53,   44,   37,   36,   68,
   49,   40,   19,   18,   67,   54,   61,   79,   69,   52,   39,   81,   50,
   73,   46,   78,   41,   89,   74,   58,   42,   56,   15,   62,   66,   47,
   12,   70,   16,   82,   71,   76,   75,   10,   88,   77,   14,   85,   92,
   83,   91,   11,   96,   84,   86,   87,   90,   98,   94,   93,   95,   97]
Length: 91, dtype: Int64


In [113]:
#checks for binning
print("<16 - ",df[df['Victim Age'] < 16].shape[0])
print("16-18 - ",df[(df['Victim Age'] >= 16) & (df['Victim Age'] < 18)].shape[0])
print("Missing  - ", df[df['Victim Age'].isna()].shape[0])

<16 -  317
16-18 -  293
Missing  -  3267


Teens between the age of 16-18 are permitted to drive if they hold a provincial driver's license in the state of California

# Cleaning age

In [114]:
df= df.dropna(subset=['Victim Age'])

In [115]:
#Creating bins for victim age
df.loc[:,'age_bin'] = pd.cut(df['Victim Age'], bins=[10, 16, 18, 30, 45, 60, np.inf],
                       labels=['Less than 16','16-18', '18-30', '30-45', '45-60', '60+'],
                       right=False, include_lowest=True)


<ipython-input-115-0005fc31f9d0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'age_bin'] = pd.cut(df['Victim Age'], bins=[10, 16, 18, 30, 45, 60, np.inf],


In [116]:
#print thecategories from age_bin excluding NaN
print(df['age_bin'].cat.categories)


Index(['Less than 16', '16-18', '18-30', '30-45', '45-60', '60+'], dtype='object')


In [117]:
df.head()

,DR Number,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,0,...,5,6,7,8,9,Intersection,Latitude,Longitude,Hour,age_bin
846,231314324,2023-07-22,08:00,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,T/C - Type of Collision,...,T/C - (A) Severe Injury,T/C - Private Property - No,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),OLIVE&WASHINGTON,34.0325,-118.2676,8,18-30
19678,240406645,2024-03-10,19:00,Hollenbeck,0605 3030 4024,31,M,C,STREET,Traffic Accident/Traffic related incident,...,None,None,None,None,None,200 W AVENUE 26,34.0786,-118.2174,19,30-45
44978,230812188,2023-07-19,12:30,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,T/C - Veh vs Veh,...,T/C - Hit and Run Misd,None,None,None,None,OLYMPIC BL&SAWTELLE BL,34.0382,-118.4412,12,30-45
45031,241307843,2024-03-27,23:40,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,Traffic Accident/Traffic related incident,...,T/C - At Intersection - Yes,T/C - PCF (A) In the Narrative,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision,ADAMS&GRAND,34.0265,-118.2723,23,18-30
45311,241214094,2024-08-05,08:35,77th Street,NaN,30,M,H,STREET,None,...,None,None,None,None,None,FLORENCE&GRAND,33.9747,-118.2803,8,30-45


In [118]:
print(df.shape)

(29847, 24)


In [119]:
#rename the final dataset
collision_data = df

###Day of Week

In [120]:
# Creates a new column with the day of the week according to the calendar
collision_data['Day'] = collision_data['Date Occurred'].dt.day_name()

collision_data["Is_Weekend"] = collision_data["Day"].isin(["Saturday", "Sunday"])

<ipython-input-120-cea3734686fb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collision_data['Day'] = collision_data['Date Occurred'].dt.day_name()
<ipython-input-120-cea3734686fb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collision_data["Is_Weekend"] = collision_data["Day"].isin(["Saturday", "Sunday"])


In [121]:
collision_data.head()

,DR Number,Date Occurred,Time Occurred,Area Name,MO Codes,Victim Age,Victim Sex,Victim Descent,Premise Description,0,...,7,8,9,Intersection,Latitude,Longitude,Hour,age_bin,Day,Is_Weekend
846,231314324,2023-07-22,08:00,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,T/C - Type of Collision,...,T/C - City Property Involved - No,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),OLIVE&WASHINGTON,34.0325,-118.2676,8,18-30,Saturday,True
19678,240406645,2024-03-10,19:00,Hollenbeck,0605 3030 4024,31,M,C,STREET,Traffic Accident/Traffic related incident,...,None,None,None,200 W AVENUE 26,34.0786,-118.2174,19,30-45,Sunday,True
44978,230812188,2023-07-19,12:30,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,T/C - Veh vs Veh,...,None,None,None,OLYMPIC BL&SAWTELLE BL,34.0382,-118.4412,12,30-45,Wednesday,False
45031,241307843,2024-03-27,23:40,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,Traffic Accident/Traffic related incident,...,T/C - Newton,T/C - Type of Collision,T/C - Movement Preceding Collision,ADAMS&GRAND,34.0265,-118.2723,23,18-30,Wednesday,False
45311,241214094,2024-08-05,08:35,77th Street,NaN,30,M,H,STREET,None,...,None,None,None,FLORENCE&GRAND,33.9747,-118.2803,8,30-45,Monday,False


In [122]:
#Checking the data type of each column
data_types = df.dtypes
data_types

,0
DR Number,int64
Date Occurred,datetime64[ns]
Time Occurred,object
Area Name,object
MO Codes,object
Victim Age,Int64
Victim Sex,object
Victim Descent,object
Premise Description,object
0,object


In [123]:
print(df.columns)  # See what columns exist
print(len(df.columns))  # Total number of columns

Index([          'DR Number',       'Date Occurred',       'Time Occurred',
                 'Area Name',            'MO Codes',          'Victim Age',
                'Victim Sex',      'Victim Descent', 'Premise Description',
                           0,                     1,                     2,
                           3,                     4,                     5,
                           6,                     7,                     8,
                           9,        'Intersection',            'Latitude',
                 'Longitude',                'Hour',             'age_bin',
                       'Day',          'Is_Weekend'],
      dtype='object')
26


Column Names will be adjusted after feature engineering to enhance readability for the rest of the project. We will also reoorder them so the seperated mo codes come after the original mo_code column

In [124]:
# adjusting column names for easier interpretation for the rest of the project
df.columns = ['Collision_ID', 'Date_Occurred', 'Time_Occurred', 'Area_Name', 'MO_Codes', 'Victim_Age', 'Victim_Sex', 'Victim_Descent', 'Premise_Description', 'MO_Code01','MO_Code02','MO_Code03','MO_Code04','MO_Code05','MO_Code06','MO_Code07','MO_Code08','MO_Code09', 'MO_Code10', 'Intersection', 'Latitude', 'Longitude', 'Hour', 'Age_Group', 'Day_of_Week', 'Is_Weekend']

#Reordering columns

df = df.iloc[:, [0,1,2,22,24,25,4,9,10,11,12,13,14,15,16,17,18,5,6,7,23,3,19,20,21,8]]

# Rechecking order
data_types = df.dtypes
data_types

,0
Collision_ID,int64
Date_Occurred,datetime64[ns]
Time_Occurred,object
Hour,int64
Day_of_Week,object
Is_Weekend,bool
MO_Codes,object
MO_Code01,object
MO_Code02,object
MO_Code03,object


In [137]:
collision_data.head()

,Collision_ID,Date_Occurred,Time_Occurred,Area_Name,MO_Codes,Victim_Age,Victim_Sex,Victim_Descent,Premise_Description,MO_Code01,...,MO_Code10,Intersection,Latitude,Longitude,Hour,Age_Group,Day_of_Week,Is_Weekend,Location_ID,Premise_ID
0,231314324,2023-07-22,08:00,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,T/C - Type of Collision,...,T/C - Central Traffic (CTD),OLIVE&WASHINGTON,34.0325,-118.2676,8,18-30,Saturday,True,1,1
1,240406645,2024-03-10,19:00,Hollenbeck,0605 3030 4024,31,M,C,STREET,Traffic Accident/Traffic related incident,...,None,200 W AVENUE 26,34.0786,-118.2174,19,30-45,Sunday,True,2,1
2,230812188,2023-07-19,12:30,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,T/C - Veh vs Veh,...,None,OLYMPIC BL&SAWTELLE BL,34.0382,-118.4412,12,30-45,Wednesday,False,3,1
3,241307843,2024-03-27,23:40,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,Traffic Accident/Traffic related incident,...,T/C - Movement Preceding Collision,ADAMS&GRAND,34.0265,-118.2723,23,18-30,Wednesday,False,4,2
4,241214094,2024-08-05,08:35,77th Street,NaN,30,M,H,STREET,None,...,None,FLORENCE&GRAND,33.9747,-118.2803,8,30-45,Monday,False,5,1


The file will now be exported after cleaning.

In [125]:
# Exports to working file
collision_data.to_csv("collision_data.csv", index=False)

# Splitting the dataset into Dimension and Fact tables

In [126]:
# Location Dimension Table
location_df = collision_data[['Intersection', 'Area_Name', 'Latitude', 'Longitude']].drop_duplicates().reset_index(drop=True)
location_df['Location_ID'] = range(1, len(location_df) + 1)

location_df = location_df.iloc[:, [4,0,1,2,3]]

# Merge Location_ID back to collision data
collision_data = collision_data.merge(location_df, on=['Intersection', 'Area_Name', 'Latitude', 'Longitude'], how='left')

collision_data.head()

,Collision_ID,Date_Occurred,Time_Occurred,Area_Name,MO_Codes,Victim_Age,Victim_Sex,Victim_Descent,Premise_Description,MO_Code01,...,MO_Code09,MO_Code10,Intersection,Latitude,Longitude,Hour,Age_Group,Day_of_Week,Is_Weekend,Location_ID
0,231314324,2023-07-22,08:00,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,T/C - Type of Collision,...,T/C - PCF (A) In the Narrative,T/C - Central Traffic (CTD),OLIVE&WASHINGTON,34.0325,-118.2676,8,18-30,Saturday,True,1
1,240406645,2024-03-10,19:00,Hollenbeck,0605 3030 4024,31,M,C,STREET,Traffic Accident/Traffic related incident,...,None,None,200 W AVENUE 26,34.0786,-118.2174,19,30-45,Sunday,True,2
2,230812188,2023-07-19,12:30,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,T/C - Veh vs Veh,...,None,None,OLYMPIC BL&SAWTELLE BL,34.0382,-118.4412,12,30-45,Wednesday,False,3
3,241307843,2024-03-27,23:40,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,Traffic Accident/Traffic related incident,...,T/C - Type of Collision,T/C - Movement Preceding Collision,ADAMS&GRAND,34.0265,-118.2723,23,18-30,Wednesday,False,4
4,241214094,2024-08-05,08:35,77th Street,NaN,30,M,H,STREET,None,...,None,None,FLORENCE&GRAND,33.9747,-118.2803,8,30-45,Monday,False,5


In [127]:
# Premise Dimension Table
premise_df = collision_data[['Premise_Description']].drop_duplicates().reset_index(drop=True)
premise_df['Premise_ID'] = range(1, len(premise_df) + 1)

premise_df = premise_df.iloc[:, [1,0]]

# Merge Premise_ID back to collision data
collision_data = collision_data.merge(premise_df, on='Premise_Description', how='left')

collision_data.head()

,Collision_ID,Date_Occurred,Time_Occurred,Area_Name,MO_Codes,Victim_Age,Victim_Sex,Victim_Descent,Premise_Description,MO_Code01,...,MO_Code10,Intersection,Latitude,Longitude,Hour,Age_Group,Day_of_Week,Is_Weekend,Location_ID,Premise_ID
0,231314324,2023-07-22,08:00,Newton,3401 3701 3036 0605 3003 3024 3033 3035 3101 4024,26,M,H,STREET,T/C - Type of Collision,...,T/C - Central Traffic (CTD),OLIVE&WASHINGTON,34.0325,-118.2676,8,18-30,Saturday,True,1,1
1,240406645,2024-03-10,19:00,Hollenbeck,0605 3030 4024,31,M,C,STREET,Traffic Accident/Traffic related incident,...,None,200 W AVENUE 26,34.0786,-118.2174,19,30-45,Sunday,True,2,1
2,230812188,2023-07-19,12:30,West LA,3004 3028 3033 3034 3037 3030,31,M,B,STREET,T/C - Veh vs Veh,...,None,OLYMPIC BL&SAWTELLE BL,34.0382,-118.4412,12,30-45,Wednesday,False,3,1
3,241307843,2024-03-27,23:40,Newton,0605 3011 3028 3033 3034 3036 3101 4013 3401 3701,25,M,O,SIDEWALK,Traffic Accident/Traffic related incident,...,T/C - Movement Preceding Collision,ADAMS&GRAND,34.0265,-118.2723,23,18-30,Wednesday,False,4,2
4,241214094,2024-08-05,08:35,77th Street,NaN,30,M,H,STREET,None,...,None,FLORENCE&GRAND,33.9747,-118.2803,8,30-45,Monday,False,5,1


In [128]:
# Fact Table: Collision
collision_fact = collision_data[['Collision_ID', 'Location_ID', 'Premise_ID']].drop_duplicates()

collision_fact.head(2)

,Collision_ID,Location_ID,Premise_ID
0,231314324,1,1
1,240406645,2,1


In [129]:
# Date Dimension Table
date_dim = collision_data[['Collision_ID', 'Date_Occurred', 'Day_of_Week']].copy()
date_dim['Year'] = collision_data['Date_Occurred'].dt.year
date_dim['Month'] = collision_data['Date_Occurred'].dt.month
date_dim['Day'] = collision_data['Date_Occurred'].dt.day

date_dim.head()

,Collision_ID,Date_Occurred,Day_of_Week,Year,Month,Day
0,231314324,2023-07-22,Saturday,2023,7,22
1,240406645,2024-03-10,Sunday,2024,3,10
2,230812188,2023-07-19,Wednesday,2023,7,19
3,241307843,2024-03-27,Wednesday,2024,3,27
4,241214094,2024-08-05,Monday,2024,8,5


In [130]:
# Time Dimension Table
time_dim = collision_data[['Collision_ID', 'Time_Occurred', 'Hour']].copy()

time_dim['Time_Occurred'] = pd.to_datetime(time_dim['Time_Occurred'], format='%H:%M', errors='coerce').dt.strftime('%H:%M')

time_dim.head()

,Collision_ID,Time_Occurred,Hour
0,231314324,08:00,8
1,240406645,19:00,19
2,230812188,12:30,12
3,241307843,23:40,23
4,241214094,08:35,8


In [131]:
# Victim Dimension Table
victim_dim = collision_data[['Collision_ID', 'Victim_Age', 'Victim_Sex', 'Victim_Descent', 'Age_Group']].copy()
victim_dim.rename(columns={'Victim_Sex': 'Victim_Gender'}, inplace=True)

victim_dim.head()

,Collision_ID,Victim_Age,Victim_Gender,Victim_Descent,Age_Group
0,231314324,26,M,H,18-30
1,240406645,31,M,C,30-45
2,230812188,31,M,B,30-45
3,241307843,25,M,O,18-30
4,241214094,30,M,H,30-45


In [132]:
# Collision MO dimension table

# Function to seperate MO codes with descriptions
def explode_mo_codes(row):
    if pd.isna(row['MO_Codes']):
      return [{
          'Collision_ID': row['Collision_ID'],
          'MO_Code': None,
          'MO_Description': 'No MO Reported'
      }]

    mo_codes = row['MO_Codes'].split()  # Assumes space-separated codes
    result = []
    for i, code in enumerate(mo_codes):
        # Dynamically get the corresponding MO description
        description = row.get(f'MO_Code{str(i+1).zfill(2)}', None)
        result.append({
            'Collision_ID': row['Collision_ID'],
            'MO_Code': code,
            'MO_Description': description
        })
    return result

# Apply the function to each row and flatten the results
mo_records = []
for _, row in collision_data.iterrows():
    mo_records.extend(explode_mo_codes(row))

# Create the final Collision_MO dimension table
collision_mo = pd.DataFrame(mo_records)

collision_mo.drop_duplicates(subset=['Collision_ID', 'MO_Code'], inplace=True)


collision_mo.head(15)

,Collision_ID,MO_Code,MO_Description
0,231314324,3401,T/C - Type of Collision
1,231314324,3701,T/C - Movement Preceding Collision
2,231314324,3036,T/C - At Intersection - Yes
3,231314324,0605,Traffic Accident/Traffic related incident
4,231314324,3003,T/C - Veh vs Ped
5,231314324,3024,T/C - (A) Severe Injury
6,231314324,3033,T/C - Private Property - No
7,231314324,3035,T/C - City Property Involved - No
8,231314324,3101,T/C - PCF (A) In the Narrative
9,231314324,4024,T/C - Central Traffic (CTD)


In [133]:
# Renaming the tables
tables = {
    'Collision': collision_fact,
    'Date': date_dim,
    'Time': time_dim,
    'Victim': victim_dim,
    'Location': location_df,
    'Premise': premise_df,
    'Collision_MO': collision_mo
}

# view all table shapes
for name, df in tables.items():
    print(f"{name} table shape: {df.shape}")


Collision table shape: (29847, 3)
Date table shape: (29847, 6)
Time table shape: (29847, 3)
Victim table shape: (29847, 5)
Location table shape: (19644, 5)
Premise table shape: (29, 2)
Collision_MO table shape: (201849, 3)


In [134]:
# Exporting all the tables as csv into a zip file

# Create a directory to save CSVs temporarily
os.makedirs("collision_tables", exist_ok=True)

# Save each DataFrame as a CSV
for name, df in tables.items():
    csv_path = f"collision_tables/{name}.csv"
    df.to_csv(csv_path, index=False)

# Create a zip file and add all CSVs
zip_filename = "collision_data_tables.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for name in tables:
        csv_path = f"collision_tables/{name}.csv"
        zipf.write(csv_path)

print(f"All tables saved and zipped into: {zip_filename}")


All tables saved and zipped into: collision_data_tables.zip
